# Problemas de incidência

Uma versão particular dos problemas de alocação são os *problemas de incidência*. Genericamente estes problemas estão ligados aos conjuntos que é possível  formar com os elementos de um dado universo finito.

É frequente usar nestes problemas uma matriz binária $A$ com a seguinte semântica:

> $A_{i,j} = 1 \quad$ se e só se $\quad$ o elemento $i$ do universo está contido no conjunto $j$.

Estas matrizes chamam-se *matrizes de incidência* e daí resulta o nome genérico para estes problemas.

Mais uma vez usaremos a biblioteca [OR-Tools](https://developers.google.com/optimization) para fazer a interface para o SCIP. Esta biblioteca pode ser instalada com o commando `pip install ortools`.

In [2]:
!pip install ortools

## Set cover

Um dos problemas clássicos das ciências da computação nesta categoria é o problema do *set cover* descrito em https://en.wikipedia.org/wiki/Set_cover_problem.

Neste problema, são dados
- $U$, o universo de valores
- $S$, o conjunto de conjuntos,  cuja união é igual a $U$

O objectivo é determinar o menor número de conjuntos de $S$ cuja união é igual a $U$, i.e., a *cobertura mínima* de $U$.

### Análise do problema

Podemos representar este problema por uma matriz de incidência $A \in \{0,1\}^{|U| \times |S|}$. Por exemplo, se $U=\{0,1,2,3,4\}$ e $S = \{\{0,1,2\},\{1,3\},\{2,3\},\{3,4\}\}$ temos a seguinte matriz de incidência

$$
\begin{array}{c|c|c|c|c}
& S_0 & S_1 & S_2 & S_3\\
\hline
0 & 1 & 0 & 0 & 0\\
\hline
1 & 1 & 1 & 0 & 0\\
\hline
2 & 1 & 0 & 1 & 0\\
\hline
3 & 0 & 1 & 1 & 1\\
\hline
4 & 0 & 0 & 0 & 1
\end{array}
$$


S0 + S3 é uma cobertura porque se juntarmos os dois temos (1) em todas as colunas. 
S0 + S1 nao é cobertura porque 4 nao pertence

Neste exemplo, a cobertura mínima é $S_0 \cup S_3 = U$.

Este problema pode ser resolvido com programação inteira usando uma variável inteira binária $x_j$ para cada conjunto $S_j$, que irá determinar se esse conjunto pertence à cobertura mínima. O objectivo é minimizar $\sum_j x_j$ obedecendo à seguinte restrição:
- Cada elemento de $U$ tem que pertencer a pelo menos um conjunto da cobertura mínima.

### Exercício 1
Formalize a restrição acima indicada

$\sum_{(0<=j<=|s|)} S_j$

$\forall_{0\le i<|U|}. \sum_{0<j<|s|} A_ij * X_j >= 1$

### Exercício 2

Usando o SCIP, implemente a função `set_cover` que dada a matriz de incidência (representada como uma lista de colunas) determine quais os conjuntos que pertencem à cobertura mínima. 

In [3]:
from ortools.linear_solver import pywraplp

def set_cover(A):
    s = pywraplp.Solver.CreateSolver('SCIP')
    # completar

    S = len(A)
    U = len(A[0])
    x = {j : s.BoolVar(str(j)) for j in range(S)}

    #Restrição
    for i in range(U):
      s.Add(sum(A[j][i] * x[j] for j in range(S)) >= 1)

    s.Minimize(sum(x.values()))

    assert(s.Solve() == pywraplp.Solver.OPTIMAL)

    return [j for j in range(S) if round(x[j].solution_value()) == 1]

assert set_cover([[1,1,1,0,0],[0,1,0,1,0],[0,0,1,1,0],[0,0,0,1,1]]) == [0,3]
            
#assert set_cover([[1,1,1,0,0],[0,1,0,1,0],[0,0,1,1,0],[0,0,0,1,1]]) == [0,3]
set_cover([[1,1,1,0,0],[0,1,0,1,0],[0,0,1,1,0],[0,0,0,1,1]])

[0, 3]

## Bin packing

Outro problema clássico na categoria dos problemas de incidência, que generaliza o problema anterior, é o problema de empacotamento *bin packing* descrito em https://en.wikipedia.org/wiki/Bin_packing_problem. 

Neste problema, são dados
- $N$, o número de items a empacotar
- $C$, a capacidade das contentores onde pretendemos empacotar os items
- $W_i$ o peso de cada item $i$, com $0 < W_i \le C$

Pretende-se determinar o número mínimo de contentores necessários para empacotar todos os items (note que, no pior caso, tal será possível com $N$ contentores).

Por exemplo, se tivermos
- $N = 7$
- $C = 10$
- $W_0 = 2, W_1 = 5, W_2 = 4, W_3 = 7, W_4 = 1, W_5 = 3, W_6 = 8$

o número mínimo de contentores necessários é 3.

### Análise do problema

Ao contrário do problema anterior, em que a matriz de incidência é dada como input, neste problema pretende-se precisamente descobrir esta matriz, minimizando simultaneamente o número de contentores. Como tal, para resolver este problema com programação inteira iremos usar as seguintes variáveis:

- Uma matriz $A$ de variáveis binárias de dimensão $|N| \times |N|$, onde a variável $A_{i,j}$ determina se o item $i$ é colocado no contentor $j$
- Uma variável binária $y_j$ por cada contentor $j$ que determina se esse contentor é utilizado

O objectivo é minimizar $\sum_j y_j$ obedecendo às seguintes restrições:
- Cada item tem que ser empacotado num contentor
- A capacidade de cada contentor não pode ser excedida

### Exercício 3
Formalize as duas restrições acima indicadas.

Min => $ \sum_{0<=j<=N} y_j$


$ \forall_{0<=i<C} \sum_{0<j<C} A_{ij} = 1 $


---$ \forall_{0<=i<N} \sum_{0<j<C} A_{ij} \le y_j $---


$ \forall_{0<=i<N} \sum_{0<j<C} A_{ij} * W_i \le C*y_i $

### Exercício 4

Implemente a função `binpacking` que dada a capacidade dos contentores e uma lista com os pesos dos items a empacotar, determine o número mínimo de contentores necessários para o fazer.

In [4]:
def binpacking(C,W):
    s = pywraplp.Solver.CreateSolver('SCIP')

    # completar
    N = len(W)
    y = { j : s.BoolVar(str(j)) for j in range(N)}
    A = { (i,j) : s.BoolVar(str(i) + ',' + str(j)) for i in range(N) for j in range(N) }

    #Restrições
    for i in range(N):
      s.Add(sum(A[i,j] for j in range(N)) == 1)

    for j in range(N):
      s.Add(sum(A[i,j] * W[i] for i in range(N)) <= C * y[j])

    s.Minimize(sum(y.values()))
    assert( s.Solve() == pywraplp.Solver.OPTIMAL)
    
    return (round(s.Objective().Value()))

binpacking(10,[2,5,4,7,1,3,8])

3

### Exercício 5
Modifique a sua implementação da função anterior por forma a devolver uma lista com os identificadores dos contentores onde cada item deve ser empacotado.

In [5]:
 def binpacking(C,W):
    s = pywraplp.Solver.CreateSolver('SCIP')

    # completar
    N = len(W)
    y = { j : s.BoolVar(str(j)) for j in range(N)}
    A = { (i,j) : s.BoolVar(str(i) + ',' + str(j)) for i in range(N) for j in range(N) }

    #Restrições
    for i in range(N):
      s.Add(sum(A[i,j] for j in range(N)) == 1)

    for j in range(N):
      s.Add(sum(A[i,j] * W[i] for i in range(N)) <= C * y[j])

    s.Minimize(sum(y.values()))
    assert( s.Solve() == pywraplp.Solver.OPTIMAL)
    
    r = []

    for i in range(N):
      for j in range(N):
        if round(A[i,j].solution_value()) == 1:
          r.append(j)
    return r

binpacking(10,[2,5,4,7,1,3,8]) 

[2, 0, 0, 1, 0, 1, 2]

## Knapsack

Outro problema clássico de incidência é o *problema da mochila* descrito em https://en.wikipedia.org/wiki/Knapsack_problem.

Neste problema, são dados
- $N$, o número de items disponíveis
- $C$, a capacidade da mochila
- $W_i$, o peso de cada item $i$, com $0 < W_i \le C$
- $V_i$ o valor de cada item $i$, com $0 \le V_i$

Pretende-se determinar o valor máximo que pode ser transportado na mochila.

Por exemplo, se tivermos uma mochila com capacidade $C = 15$ e 5 items com os seguintes pesos e valores
$$
\begin{array}{c|c|c}
& W & V\\
\hline
0 & 12 & 4\\
1 & 2 & 2\\
2 & 1 & 2\\
3 & 1 & 1\\
4 & 4 & 10
\end{array}
$$
a melhor solução é empacotar todos os items menos o primeiro, com um valor total de 15.

### Exercício 6
Formalize este problema usando programação inteira.

$ \sum_i x_i * w_i \le C$

Maximizar o lucro $ \sum_i x_i * v_i$

### Exercício 7
Implemente a função `knapsack` que dada a capacidade da mochila e uma lista com um par *(peso, valor)* por cada item, determine quais os items a empacotar na mochila e o respectivo valor.

In [28]:
def knapsack(C,I):
    s = pywraplp.Solver.CreateSolver('SCIP')
    # completar
    N = len(I)
    W = {i: s.BoolVar(str(i)) for i in range(N) }
    
    s.Add(sum(W[i] * I[i][0] for i in range(N))<=C)
            
    s.Maximize(sum(W[i] * I[i][1] for i in range(N)))   
    
    assert(s.Solve() == pywraplp.Solver.OPTIMAL)
    r = []
    for i in range(N):
        for j in range(N): 
            if round(W[i].solution_value()) == 1:
                r.append(j)
    return r 
knapsack(15,[(12,4),(2,2),(1,2),(1,1),(4,10)])

[0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4]